# Web scraping - Managers

In [22]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
import json

In [23]:
load_dotenv()

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

I will be scraping manager details from https://www.premierleague.com/managers?se=210&cl=-1, which needed the api which was https://footballapi.pulselive.com/football/teamofficials?pageSize=500&comps=1&altIds=true&type=manager&compCodeForActivePlayer=EN_PR&page=0

In [52]:
page = requests.get('https://footballapi.pulselive.com/football/teamofficials?pageSize=500&comps=1&altIds=true&type=manager&compCodeForActivePlayer=EN_PR&page=0',headers = {
        "origin": "https://www.premierleague.com"
    }).text
managers = json.loads(page)

In [53]:
managers = managers['content']
managers[0:2]
    
    

[{'officialId': 89.0,
  'role': 'Manager',
  'currentTeam': {'name': 'Tottenham Hotspur',
   'club': {'name': 'Tottenham Hotspur',
    'shortName': 'Spurs',
    'abbr': 'TOT',
    'id': 21.0},
   'teamType': 'FIRST',
   'shortName': 'Spurs',
   'id': 21.0,
   'altIds': {'opta': 't6'}},
  'active': False,
  'birth': {'date': {'millis': -791683200000.0, 'label': '30 November 1944'},
   'country': {'isoCode': 'GB-SCT',
    'country': 'Scotland',
    'demonym': 'Scottish'}},
  'age': '76 years 211 days',
  'name': {'display': 'George Graham', 'first': 'George', 'last': 'Graham'},
  'id': 30.0,
  'altIds': {'opta': 'man37371'}},
 {'officialId': 29.0,
  'role': 'Manager',
  'currentTeam': {'name': 'Everton',
   'club': {'name': 'Everton',
    'shortName': 'Everton',
    'abbr': 'EVE',
    'id': 7.0},
   'teamType': 'FIRST',
   'shortName': 'Everton',
   'id': 7.0,
   'altIds': {'opta': 't11'}},
  'active': False,
  'birth': {'date': {'millis': -760320000000.0, 'label': '28 November 1945'},
 

In [48]:
managers_list = []
for manager in managers:
    manager_row = [manager["name"]["first"],manager["name"]["last"],
                   manager["birth"]["country"]["isoCode"],
                   manager["birth"]["date"]["label"]]
    managers_list.append(manager_row)
df = pd.DataFrame(managers_list,columns=["first_name","surname","nationality","date_of_birth"])
df["first_name"] = df["first_name"].str.lower().str.strip()
df["surname"] = df["surname"].str.lower().str.strip()
df["date_of_birth"] = pd.to_datetime(df["date_of_birth"])
df["role"] = "manager"
df["nationality"] = df["nationality"].str.lower().str.strip()
df

,first_name,surname,nationality,date_of_birth,role
0,george,graham,gb-sct,1944-11-30,manager
1,mike,walker,gb-wls,1945-11-28,manager
2,ian,porterfield,gb-eng,1946-02-11,manager
3,joe,royle,gb-eng,1949-04-08,manager
4,bobby,gould,gb-wls,1946-01-12,manager
...,...,...,...,...,...
236,mikel,arteta,es,1982-03-26,manager
237,marcelo,bielsa,ar,1955-07-21,manager
238,thomas,tuchel,de,1973-08-29,manager
239,paul,heckingbottom,gb-eng,1977-07-17,manager


In [49]:
df.to_csv("../CSVs/managers.csv")